# Strings Homework

In [1]:
import numpy as np
from astropy.table import QTable, Column
from astroquery.gaia import Gaia
import pandas as p

## Copy and Paste the Gaia query you used in the last homework to get data from the zenith.

In [2]:
query = """
SELECT TOP 100
source_id, ra, dec, phot_g_mean_mag, bp_rp, parallax
FROM gaiadr3.gaia_source_lite
WHERE
    DISTANCE( POINT(119.8, 47.0), POINT(ra, dec) ) < 1.0
    AND phot_g_mean_mag < 10.0
    AND parallax > 0.0
    AND bp_rp IS NOT NULL
ORDER BY bp_rp ASC
"""

In [3]:
job_query = Gaia.launch_job(query)
print(job_query)

<Table length=36>
      name       dtype  unit                            description                            
--------------- ------- ---- ------------------------------------------------------------------
      source_id   int64      Unique source identifier (unique within a particular Data Release)
             ra float64  deg                                                    Right ascension
            dec float64  deg                                                        Declination
phot_g_mean_mag float32  mag                                              G-band mean magnitude
          bp_rp float32  mag                                                     BP - RP colour
       parallax float64  mas                                                           Parallax
Jobid: None
Phase: COMPLETED
Owner: None
Output file: 1706056798201O-result.vot.gz
Results: None


In [4]:
star_table = job_query.get_results()
star_table

source_id,ra,dec,phot_g_mean_mag,bp_rp,parallax
,deg,deg,mag,mag,mas
int64,float64,float64,float32,float32,float64
932851782702640128,121.1837299677284,47.28846093314972,9.224431,0.59405327,3.9774703932418434
932376931117076608,119.35794145891437,46.3384471259334,9.816679,0.67705536,6.234940232318302
932471420398832000,119.9196815899295,46.61987911083302,8.016949,0.6863818,6.957786644888596
932931359856644224,120.55034374781623,47.58860439361576,9.678056,0.692132,6.90085897359672
933103811383007744,119.9957105955765,47.78505238304345,8.517405,0.7436218,9.864936527454228
932764715125240576,118.8294649241317,47.42740866769713,9.089646,0.7580681,2.2550859647279298
932778562099849856,119.03617666610461,47.55034822924161,9.888519,0.895524,10.587155139199597
932384043584096384,119.5069805080389,46.43745623074234,8.470822,1.1134033,2.4361439091131536


## Add a column `distance` to your table that has the distance (in parsecs) for all of the objects

In [5]:
def dpc(p):
    return (1 / (p/1000))

In [6]:
distance_values = [dpc(p = parallax_value) for parallax_value in star_table['parallax']]
distance_column = Column(distance_values, name='distance', dtype='float')
star_table.add_column(distance_column)

star_table

source_id,ra,dec,phot_g_mean_mag,bp_rp,parallax,distance
,deg,deg,mag,mag,mas,
int64,float64,float64,float32,float32,float64,float64
932851782702640128,121.1837299677284,47.28846093314972,9.224431,0.59405327,3.9774703932418434,251.41607633311597
932376931117076608,119.35794145891437,46.3384471259334,9.816679,0.67705536,6.234940232318302,160.38646125532716
932471420398832000,119.9196815899295,46.61987911083302,8.016949,0.6863818,6.957786644888596,143.7238666601872
932931359856644224,120.55034374781623,47.58860439361576,9.678056,0.692132,6.90085897359672,144.90949660413088
933103811383007744,119.9957105955765,47.78505238304345,8.517405,0.7436218,9.864936527454228,101.3691266250917
932764715125240576,118.8294649241317,47.42740866769713,9.089646,0.7580681,2.2550859647279298,443.4420752206878
932778562099849856,119.03617666610461,47.55034822924161,9.888519,0.895524,10.587155139199597,94.45408014259073
932384043584096384,119.5069805080389,46.43745623074234,8.470822,1.1134033,2.4361439091131536,410.48478140359


----
## For each question below:

- Use `f-string` formatting to make your output look nice
- All floats should be printed to one decimal place
- Replace the CAP values with data. **DO NOT HARDCODE!!** - use variables.

## Print the following sentence for the nearest object

`The object SOURCE_ID is the nearest object with a parallax of PARALLAX mas, and a distance of DISTANCE pc.`

In [7]:
nearest = star_table[star_table['distance'] == star_table['distance'].min()]
n_source = nearest[0]['source_id']
n_parallax = nearest[0]['parallax']
n_distance = nearest[0]['distance']

In [21]:
print(f"The object {n_source} is the nearest object with a parallax of {n_parallax:.1f} mas, and a distance of {n_distance:.1f} pc.")

The object 932778562099849856 is the nearest object with a parallax of 10.6 mas, and a distance of 94.5 pc.


## Print the following sentence for the brightest object

`The object SOURCE_ID is the brightest object with a G magnitude of PHOT_G_MEAN_MAG mag, and a distance of DISTANCE pc.`

In [9]:
brightest = star_table[star_table['phot_g_mean_mag'] == star_table['phot_g_mean_mag'].min()]
b_source = brightest[0]['source_id']
b_magnitude = brightest[0]['phot_g_mean_mag']
b_distance = brightest[0]['distance']

In [22]:
print(f"The object {b_source} is the brightest object with a G magnitude of {b_magnitude:.1f} mag, and a distance of {b_distance:.1f} pc.")

The object 932769353689866752 is the brightest object with a G magnitude of 4.9 mag, and a distance of 219.7 pc.


## And a summary statement:

`There are NUMBER objects in the table. The objects have an average distance of AVE_DISTANCE pc, and a range in G magnitudes from G_MAX_MAG to G_MIN_MAG.`

In [19]:
ave_distance = np.mean(star_table['distance'])
g_max_mag = star_table['phot_g_mean_mag'].min()
g_min_mag = star_table['phot_g_mean_mag'].max()
number = len(star_table)

In [25]:
print(f"There are {number} objects in the table. The objects have an average distance of {ave_distance:.1f} pc, and a range in G magnitudes from {g_max_mag:.1f} to {g_min_mag:.1f}.")

There are 36 objects in the table. The objects have an average distance of 717.9 pc, and a range in G magnitudes from 4.9 to 9.9.


---

### Due Wed Jan 24 - 1pm
- `Save notebook as .html: File -> Download as -> HTML`
- `Upload .html to Canvas -> Strings assignment`